In [8]:
from z3 import *

# Function to read the .dat file and return the parameters
def read_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse m (number of couriers) and n (number of items)
    num_couriers = int(lines[0].strip())
    num_objects = int(lines[1].strip())

    # Parse load limits
    capacities = list(map(int, lines[2].strip().split()))

    # Parse item sizes
    weights = list(map(int, lines[3].strip().split()))

    # Parse the distance matrix
    distance_matrix = []
    for i in range(num_objects + 1):  # n+1 because the last row is the origin
        distance_matrix.append(list(map(int, lines[4 + i].strip().split())))

    return num_couriers, num_objects, capacities, weights, distance_matrix

In [11]:
from z3 import *
from itertools import combinations
def solve_mcp(file_path):
    # Read instance data from .dat file
    num_couriers, num_objects, capacities, weights, distance_matrix = read_instance(file_path)
    # Assuming we already have `num_couriers`, `num_objects`, `capacities`, `weights`, and `distance_matrix`
    depot = num_objects # Index for the depot (last element in distance matrix)

    # Variables
    load = [Int(f'load_{i}') for i in range(num_couriers)]
    assignments = [[Bool(f'assignments_{i}_{j}') for j in range(num_objects)] for i in range(num_couriers)]

    # Routing variables
    # route[i][j][k] == True if courier k travels directly from point i to point j
    route = [[[Bool(f'route_{i}_{j}_{k}') for k in range(num_couriers)] for j in range(len(distance_matrix))] for i in range(len(distance_matrix))]
    visit_order = [[Int(f'visit_order_{j}_{k}') for j in range(num_objects + 1)] for k in range(num_couriers)]

    # SMT Solver
    solver = Optimize()

    # Constraint: Each item should be assigned to exactly one courier
    for j in range(num_objects):
        solver.add(Or([assignments[k][j] for k in range(num_couriers)]))  # At least one assignment
        for k1, k2 in combinations(range(num_couriers), 2):
            solver.add(Or(Not(assignments[k1][j]), Not(assignments[k2][j])))  # At most one assignment

    # Load and capacity constraints for each courier
    for i in range(num_couriers):
        # Each load is the sum of weights assigned to the courier
        solver.add(load[i] == Sum([If(assignments[i][j], weights[j], 0) for j in range(num_objects)]))
        solver.add(load[i] <= capacities[i])

    # Channeling constraints between assignments and routes
    for k in range(num_couriers):
        for j in range(num_objects):
            # If item `j` is assigned to courier `k`, courier `k` must visit `j`
            solver.add(Implies(assignments[k][j], Or([route[i][j][k] for i in range(num_objects + 1) if i != j])))

            # If there's a route to item `j` for courier `k`, item `j` must be assigned to courier `k`
            solver.add(Implies(Or([route[i][j][k] for i in range(num_objects + 1) if i != j]), assignments[k][j]))

    # Additional Constraints

    # 1. Ensure each courier starts from and returns to the depot
    for k in range(num_couriers):
        # Courier `k` should start at the depot, with exactly one route from the depot to some other point
        solver.add(Sum([If(route[depot][j][k], 1, 0) for j in range(num_objects)]) == 1)

        # Courier `k` should return to the depot, with exactly one route to the depot from some other point
        solver.add(Sum([If(route[j][depot][k], 1, 0) for j in range(num_objects)]) == 1)

    # Flow control to prevent subtours and ensure continuous paths
    for k in range(num_couriers):
        solver.add(visit_order[k][depot] == 0)  # Depot is the starting point

        for j in range(num_objects + 1):
            # Ensure that each non-depot location (assigned location) has exactly one incoming and one outgoing route if visited
            if j != depot:
                solver.add(Sum([If(route[i][j][k], 1, 0) for i in range(num_objects + 1) if i != j]) == If(assignments[k][j], 1, 0))
                solver.add(Sum([If(route[j][i][k], 1, 0) for i in range(num_objects + 1) if i != j]) == If(assignments[k][j], 1, 0))

        # Enforce continuity in visit order if there is a route from i to j for courier k
        for i in range(num_objects + 1):
            for j in range(num_objects + 1):
                if i != j and j != depot:
                    solver.add(Implies(route[i][j][k], visit_order[k][j] == visit_order[k][i] + 1))

    # Objective: Minimize the maximum distance traveled by any courier
    max_distance = Int("max_distance")
    total_distance = [Int(f"total_distance_{i}") for i in range(num_couriers)]
    for k in range(num_couriers):
        # Sum distances for the route of each courier
        solver.add(total_distance[k] == Sum([If(route[i][j][k], distance_matrix[i][j], 0)
                                            for i in range(len(distance_matrix))
                                            for j in range(len(distance_matrix))]))
        solver.add(total_distance[k] <= max_distance)  # Ensure max_distance bounds each courier's total distance

    solver.minimize(max_distance)  # Minimize the maximum distance among all couriers

# Solve and output
    if solver.check() == sat:
        model = solver.model()
        loads = [model.evaluate(load[i]) for i in range(num_couriers)]
        assignments_values = [[model.evaluate(assignments[i][j]) for j in range(num_objects)] for i in range(num_couriers)]
        distances = [model.evaluate(total_distance[i]) for i in range(num_couriers)]
        print("Loads per courier:", loads)
        print("Assignments:", assignments_values)
        print("Distances per courier:", distances)
        print("Max distance:", model.evaluate(max_distance))

        # Output route details
        for k in range(num_couriers):
            print(f"Courier {k + 1}:")
            for i in range(len(distance_matrix)):
                for j in range(len(distance_matrix)):
                    # Use bool() to force conversion to a concrete Boolean value
                    if is_true(model.evaluate(route[i][j][k])):  # Cast symbolic value to Boolean
                        print(f"  Point {i + 1} -> Point {j + 1}")

    else:
        print("No solution found.")



In [12]:
import os
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError

# Directory containing instance files
instance_folder = '/Users/shariqansari/Documents/CDMO/instances'

# Maximum allowed time for each instance (in seconds)
timeout_per_instance = 300

# Function to solve MCP problem for all files in the instance folder with a per-instance timeout
def solve_all_instances():
    # List and sort files in ascending order
    files = sorted([f for f in os.listdir(instance_folder) if f.endswith(".dat")])

    with ThreadPoolExecutor(max_workers=1) as executor:  # Using a single worker to keep order
        for filename in files:
            file_path = os.path.join(instance_folder, filename)
            print(f"Processing file: {file_path}")
            
            # Submit the solve_mcp task with a timeout
            future = executor.submit(solve_mcp, file_path)
            
            try:
                # Wait for the result with the specified timeout
                future.result(timeout=timeout_per_instance)
            except TimeoutError:
                print(f"Timeout: {file_path} took longer than {timeout_per_instance} seconds and was skipped.")
            except Exception as e:
                print(f"An error occurred while processing {file_path}: {e}")

# Call the function to start processing files
solve_all_instances()



Processing file: /Users/shariqansari/Documents/CDMO/instances/inst01.dat
Loads per courier: [14, 10]
Assignments: [[True, False, True, True, False, False], [False, True, False, False, True, True]]
Distances per courier: [14, 14]
Max distance: 14
Courier 1:
  Point 1 -> Point 7
  Point 3 -> Point 1
  Point 4 -> Point 3
  Point 7 -> Point 4
Courier 2:
  Point 2 -> Point 5
  Point 5 -> Point 6
  Point 6 -> Point 7
  Point 7 -> Point 2
Processing file: /Users/shariqansari/Documents/CDMO/instances/inst02.dat
Loads per courier: [15, 2, 27, 11, 24, 43]
Assignments: [[False, False, False, False, False, True, False, True, False], [False, False, False, False, True, False, False, False, False], [True, False, False, True, False, False, False, False, False], [False, True, False, False, False, False, False, False, False], [False, False, False, False, False, False, True, False, False], [False, False, True, False, False, False, False, False, True]]
Distances per courier: [199, 192, 209, 226, 116, 189]

KeyboardInterrupt: 